In [55]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_hub as hub

In [56]:
url  = 'https://tfhub.dev/google/imagenet/mobilenet_v2_130_224/feature_vector/5'

In [57]:
training_data_path = './seg_train/seg_train'
test_data_path = "./seg_test/seg_test"

In [58]:
IMAGE_SIZE = (150, 150)
BATCH_SIZE = 64

In [59]:
data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center= True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=  0.2,
    horizontal_flip = True
)

In [60]:
train_data = data_gen.flow_from_directory(
    training_data_path,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed = 42,
    class_mode = 'categorical'
)
test_data = data_gen.flow_from_directory(
    test_data_path,
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    shuffle=True,
    seed = 42,
    class_mode = 'categorical'
)


Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [61]:
base_model = tf.keras.applications.InceptionV3(include_top=True)

In [64]:
base_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 149, 149, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 149, 149, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [65]:
base_input = base_model.layers[0].input
#base_input.trainable = False
feature_vector = base_model.layers[-2].output 

In [66]:
#output = tf.keras.layers.Dense(1000)(feature_vector)
output = tf.keras.layers.Dense(6,activation = 'softmax')(feature_vector)

In [67]:
model = None
model = tf.keras.Model(inputs = base_input , outputs = output)

In [68]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)


In [69]:
history = model.fit(train_data,epochs = 50 , validation_data=test_data, batch_size = BATCH_SIZE)

Epoch 1/50
439/439 [==============================] - 371s 803ms/step - loss: 0.6214 - accuracy: 0.7850 - val_loss: 0.6348 - val_accuracy: 0.7727
Epoch 2/50
439/439 [==============================] - 139s 315ms/step - loss: 0.4548 - accuracy: 0.8416 - val_loss: 0.5923 - val_accuracy: 0.7983
Epoch 3/50
439/439 [==============================] - 134s 306ms/step - loss: 0.4028 - accuracy: 0.8578 - val_loss: 0.5761 - val_accuracy: 0.8053
Epoch 4/50
439/439 [==============================] - 141s 322ms/step - loss: 0.3823 - accuracy: 0.8663 - val_loss: 0.4619 - val_accuracy: 0.8210
Epoch 5/50
439/439 [==============================] - 283s 646ms/step - loss: 0.3430 - accuracy: 0.8809 - val_loss: 0.7203 - val_accuracy: 0.7547
Epoch 6/50
439/439 [==============================] - 120s 273ms/step - loss: 0.3354 - accuracy: 0.8828 - val_loss: 0.4119 - val_accuracy: 0.8550
Epoch 7/50
439/439 [==============================] - 104s 237ms/step - loss: 0.3176 - accuracy: 0.8865 - val_loss: 0.6714 -